# **Proyecto final**

---

## **Objetivo**
---

Este proyecto tiene como objetivo desarrollar el análisis de una muestra de viajes de una de las empresas de servicios de transporte privado más reconocidas del mundo. Mediante dicho análisis se busca preparar una Tabla Análitica de Datos (TAD) que permita la correcta implementación de modelos que predigan el precio ideal de un viaje para los usuarios que buscan transportarse de manera segura y con conductores confiables.

El dataset contiene las siguientes variables:

- **key**: identificador único para cada viaje.

- **fare_amount**: precio de cada viaje en dólares.

- **pickup_datetime**: fecha y hora en que se activó el medidor.

- **passenger_count**: el número de pasajeros en el vehículo (valor ingresado por el conductor).

- **pickup_longitude**: la geolocalización (longitud) en la que se activó el medidor.

- **pickup_latitude**: la geolocalización (latitud) en la que se activó el medidor.

- **dropoff_longitude**: la longitud en la que se desconectó el medidor.

- **dropoff_latitude**: la latitud donde se desconectó el medidor.

## **Fuente**
---
**Datos**: Kaggle.

## **Datos**
---
**Nombre:** Alan Ruiz Mondragón.  
**Grupo:** 18.

## **Librerias**
---

In [ ]:
#Importamos librerias
from  functools import reduce
from  scipy.stats  import  normaltest
from category_encoders.count import CountEncoder
from plotly.offline import plot,iplot
from scipy import stats
from scipy.stats import chisquare
from scipy.stats import ksone
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from varclushi import VarClusHi
import cufflinks as cf
import datetime
import emoji
import jellyfish as jf
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re 
import seaborn as sns
import unicodedata
import warnings
import datetime as dt
warnings.filterwarnings("ignore")
import geopy.distance
from IPython.display import Image
import urllib
import plotly.express as px
import geopandas as gpd


#Definimos configuraciones
cf.go_offline()
pd.set_option("display.max_columns",200)
pd.set_option("display.max_rows",200)

## **Funciones**
---

In [ ]:
#Definimos una función para limpieza de texto
def clean_text(text, pattern="[^a-zA-Z0-9]"):
    text=str(text)
    cleaned_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore')
    cleaned_text = re.sub(pattern, " ", cleaned_text.decode("utf-8"), flags=re.UNICODE)
    cleaned_text = u' '.join(cleaned_text.lower().strip().lstrip().split())
    return cleaned_text if cleaned_text!="nan" else np.nan

#Definimos una función para revisar la completitud de nuestras variables
def completitud(df):
    comple=pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace=True)
    comple=comple.rename(columns={"index":"columna",0:"total"})
    comple["completitud"]=(1-comple["total"]/df.shape[0])*100
    comple=comple.sort_values(by="completitud",ascending=True)
    comple.reset_index(drop=True,inplace=True)
    return comple

#Definimos funciones para revisar las variables unitarias
def unitarias(df,col):
    result=pd.DataFrame(df[col].value_counts(1))
    if result.shape[0]>0:
        if (result[col].values[0]>.91) :
            print(f"{col} -- VARIABLE UNITARIA")

def categoricas(df,col):
    result=pd.DataFrame(df[col].value_counts(1))
    if result.shape[0]>0:
        if (result[col].values[0]>.91) :
            print(f"{col} -- VARIABLE UNITARIA")
        result[col]=result[col].map(lambda x:str(round(x*100,2))+"%")
        result.reset_index(inplace=True)
        result.columns=[col+"_valores","%_aparicion"]
    return result

#Definimos funciones para realizar visualizaciones
def bar(df,col,title,x_title="",y_title=""):
    layout = go.Layout(font_family="JetBrains Mono, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"text": x_title,"font": {"family": 'JetBrains Mono, monospace',"size": 18,
        "color": '#000000'}}},yaxis= {"title": {"text": y_title,"font": {"family": 'JetBrains Mono monospace',"size": 18,
        "color": '#000000'}}},title_font_family="JetBrains Mono, monospace",title_font_color="#000000",template="plotly_white")
    aux=pd.DataFrame(df[col].value_counts()).reset_index().rename(columns={"index":"conteo"})
    fig=aux.iplot(kind='bar',x="conteo",y=col,title=title,asFigure=True,barmode="overlay",sortbars=True,color="#000000",layout=layout)
    fig.update_layout(width=800)
    fig.update_traces(marker_color='#005a96')
    return fig

def pie(df,col,title,x_title="",y_title=""):
    layout = go.Layout(font_family="JetBrains Mono, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"text": x_title,"font": {"family": 'JetBrains Mono, monospace',"size": 18,
        "color": '#000000'}}},yaxis= {"title": {"text": y_title,"font": {"family": 'JetBrains Mono monospace',"size": 18,
        "color": '#000000'}}},title_font_family="JetBrains Mono, monospace",title_font_color="#000000",template="plotly_white")
    #layout = go.Layout(template="plotly_white")
    colors=[ "#581845", "#900c3f","#c70039","#ff5733","#ffc305","#005ba3","#0061a9","#1567af","#226cb6","#2c72bc", "#0061a9","#4c79b7","#7492c6","#98acd4","#bbc7e2","#dde3f1","#ffffff"
]
    aux=pd.DataFrame(df[col].value_counts()).reset_index().rename(columns={"index":"conteo"})
    fig=aux.iplot(kind='pie',labels="conteo",values=col,title=title,asFigure=True,theme="white")
    
    fig.update_traces(textfont_size=10,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    return fig
    
def box(df,col,title):
    layout = go.Layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"font": {"family": 'Courier New, monospace',"size": 18,
        "color": '#002e4d'}}},title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    fig=df[[col]].iplot(kind='box',title=title,asFigure=True,theme="white",layout=layout,color="#005a96", boxpoints='outliers')
    return fig

def histogram(df,col,bins,title):
    layout = go.Layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"font": {"family": 'Courier New, monospace',"size": 18,
        "color": '#002e4d'}}},title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    fig=df[[col]].iplot(kind='histogram',x=col,bins=bins,title=title,asFigure=True,theme="white",layout=layout,color="#003e6c")
    fig.update_traces(opacity=0.90)
    return fig

#Definimos funciones para la revisión de outliers
def OUTLIERS(df,cols):
    results=pd.DataFrame()
    data_iqr=df.copy()
    data_per=df.copy()
    total=[]
    total_per=[]
    total_z=[]
    indices_=[]

    for col in cols:
        #IQR
        Q1=df[col].quantile(0.25)
        Q3=df[col].quantile(0.75)
        IQR=Q3-Q1
        INF=Q1-1.5*(IQR)
        SUP=Q3+1.5*(IQR)
    
        
        n_outliers=df[(df[col] < INF) | (df[col] > SUP)].shape[0]
        total.append(n_outliers)
        indices_iqr=list(df[(df[col] < INF) | (df[col] > SUP)].index)
        #data_iqr=data_iqr[~(data_iqr[col] < INF) | (data_iqr[col] > SUP)].reset_index(drop=True)
        
        #Percentiles
        INF_pe=np.percentile(df[col].dropna(),5)
    
        SUP_pe=np.percentile(df[col].dropna(),95)
        n_outliers_per=df[(df[col] < INF_pe) | (df[col] > SUP_pe)].shape[0]
        total_per.append(n_outliers_per)
        indices_per=list(df[(df[col] < INF_pe) | (df[col] > SUP_pe)].index)
        #data_per=data_per[~(data_per[col] < INF_pe) | (data_per[col] > SUP_pe)].reset_index(drop=True)
        
        #MEAN CHANGE
        
        #Obtenemos todos los percentiles además del máximo
        perc_100 = [x / 100 for x in range(100)]
        dist = df[col].describe(perc_100).iloc[4:]
        #Obtenemos el cambio entre percentiles
        change_dist = df[col].describe(perc_100).iloc[4:].diff()
        #Obtenemos el cambio promedio entre percentiles
        mean_change = df[col].describe(
            perc_100).iloc[4:].diff().mean()
        #Si el cambio entre el percentil 99 y el maximo es mayor a el cambio promedio entonces:
        if change_dist["max"] > mean_change:
            #La banda superior será el máximo menos el cambio promedio
            ub = dist["max"] - mean_change
            #si la banda superior es más pequeña que el percentil 99 , modificamos la banda para que tome el percentil 99
            if ub < dist["99%"]:
                ub = dist["99%"]
        else:
        #Si el cambio entre el percentil 99 y el maximo es menor o igual a el cambio promedio entonces se toma el percentil 99
            ub = dist["max"]

        if change_dist["1%"] > mean_change:
            lb = dist["0%"] + mean_change
            if lb > dist["1%"]:
                lb = dist["1%"]
        else:
            lb = dist["0%"]
        n_total_z=df[(df[col] < lb) | (df[col] > ub)].shape[0]
        total_z.append(n_total_z)
        indices_z=list(df[(df[col] < lb) | (df[col] > ub)].index)
        
        indices_.append(aux_outliers(indices_iqr,indices_per,indices_z))

    results["features"]=cols
    results["n_outliers_IQR"]=total
    results["n_outliers_Percentil"]=total_per
    results["n_outliers_Mean_Change"]=total_z
    results["n_outliers_IQR_%"]=round((results["n_outliers_IQR"]/df.shape[0])*100,2)
    results["n_outliers_Percentil_%"]=round((results["n_outliers_Percentil"]/df.shape[0])*100,2)
    results["n_outliers_Mean_Change_%"]=round((results["n_outliers_Mean_Change"]/df.shape[0])*100,2)
    results["indices"]=indices_
    results["total_outliers"]=results["indices"].map(lambda x:len(x))
    results["%_outliers"]=results["indices"].map(lambda x:round(((len(x)/df.shape[0])*100),2))
    results=results[['features', 'n_outliers_IQR', 'n_outliers_Percentil',
       'n_outliers_Mean_Change', 'n_outliers_IQR_%', 'n_outliers_Percentil_%',
       'n_outliers_Mean_Change_%',  'total_outliers', '%_outliers','indices']]
    return results
    
def aux_outliers(a,b,c):
    a=set(a)
    b=set(b)
    c=set(c)
    
    a_=a.intersection(b)

    b_=b.intersection(c)

    c_=a.intersection(c)

    outliers_index=list(set(list(a_)+list(b_)+list(c_)))
    return outliers_index

#Definimos funciones para la revisión de outliers el método a utilizar
def chi_square(df,col,valor_miss):
    x_i=df[col].fillna(valor_miss).value_counts()
    k=x_i.sum()
    p_i=df[col].dropna().value_counts(1)
    m_i=k*p_i
    print(x_i)
    print(m_i)
    chi=chisquare(f_obs=x_i,f_exp=m_i)
    p_val=chi.pvalue
    alpha=0.05
    if p_val<alpha:
        print("Rechazamos HO (La porporción de categorias es la misma que la general)")
    else:
        print("Aceptamos HO (La porporción de categorias es la misma que la general)")

## **Dataset**
---

In [ ]:
#Cargamos los datos
df = pd.read_csv('/Users/alanruizmondragon/Documents/PERSONAL/Code/Diplomados/Ciencia de datos/Modulo I/proyecto_final/data/uber.csv')

In [ ]:
#Visualizamos el dataset
df.head()

In [ ]:
#Eliminamos la columna desconocida y la variable key debido a que esta variable corresponde alos mismos datos de la variable pickup_datetime
df.drop(columns={'Unnamed: 0','key'}, axis=1, inplace=True)
#Visualizamos nuevamente el dataset
df.head()

In [ ]:
#Modificamos la variable de fecha a un tipo datetime para su posterior manipulación
df.pickup_datetime = pd.to_datetime(df.pickup_datetime)

In [ ]:
#Visualizamos la información inicial del dataset dataset
print('Visualizamos el dataset')
display(df.head())

**Tamaño**

In [ ]:
#Visualizamos el tamaño del dataset
print('El dataset contienene {} columnas y {} registros.'.format(df.shape[1], "{:,}".format(int(df.shape[0]))))

**Información**

In [ ]:
#Obtenemos información del dataset
print('Obtenemos la información del dataset\n')
df.info()

In [ ]:
#Visualizamos el número de columnas por tipo
df.dtypes.value_counts()


**Diccionario**

In [ ]:
#Cargamos el diccionario de datos del dataset principal
diccionario = pd.read_excel('/Users/alanruizmondragon/Documents/PERSONAL/Code/Diplomados/Ciencia de datos/Modulo I/proyecto_final/diccionario/diccionario.xlsx')

In [ ]:
#Visualizamos el diccionario de datos del dataset principal 
diccionario

## **Calidad de datos**
---

### Definición

La calidad de datos se refiere al grado en que los datos se ajustan a los criterios establecidos para su uso. Estos criterios de calidad de los datos abarcan aspectos como exactitud, coherencia, actualización, exhaustividad y decirigibilidad. La calidad de los datos es uno de los principales factores en el éxito de la base de datos. Si los datos no cumplen con los criterios de calidad adecuados, la información generada a partir de dichos datos también será maliciosa o inútil. Por lo tanto, es crítica la necesidad de vigilancia y mejora de la calidad de los datos.

In [ ]:
#Guardamos el dataset original antes de aplicar la calidad de datos
original_df = df.copy()

### Etiquetado de variables

En este aparatdo se realizará la revisión del dataset y se etiquetarán las variables para identificarlas a lo largo dle tratamiento de datos.

#### Contenido

Visualizamos el contenido y las columnas de los datasets nuevamente para registrar los siguientes pasos de calidad.

In [ ]:
#Visualizamos el dataset
display(df.head())
display(df.shape)

In [ ]:
#Visualizamos las columnas contenidas dentro del dataset principal
df.columns

In [ ]:
#Visualizamos el contenido del datset principal
df.info()

In [ ]:
#Visualizamos la distribución del contenido por tipos de datos de la columna del dataset principal 
df.dtypes.value_counts()

##### Distribución del contenido

Los tipos de datos contenidos en el dataset se distribuyen de la siguiente manera

- Columnas de tipo float64: 5
- Columnas de tipo int64: 1
- Columnas de tipo object: 1

#### Prefijos

Realizamos el etiquetado de las variables utilizando los siguientes prefijos que utilizaremos para los diferentes tipos de variables:

- **c_**: Variables numericas (discretas y continuas).
- **v_**: Variables categoricas.
- **d_**: Variables tipo fecha.
- **t_**: Variables de texto (comentarios, descripciones, url, etc.).
- **g_**: Variables geograficas.

#### Etiquetado

**Nota**: Es importante mencionar que la variable target es la variable *****fare_amount*****, ya que el objetivo posterior al análisis es realizar un modelo de aprendizaje supervisado que nos permita predecir el precio de un viaje.

In [ ]:
#Realizamos el etiquetado de las variables del datset principal de acuerdo a los prefijos anteriormente establecidos
c_feats=["fare_amount", "pickup_longitude", "pickup_latitude", "dropoff_longitude", "dropoff_latitude"]
v_feats=["passenger_count"] 
d_feats=["pickup_datetime", ]

In [ ]:
#Aplicamos 
c_feats_new = ["c_" + x for x in c_feats]
v_feats_new = ["v_" + x for x in v_feats]
d_feats_new = ["d_" + x for x in d_feats]

#Renombramos las columnas
df.rename(columns = dict(zip(c_feats, c_feats_new)), inplace = True)
df.rename(columns = dict(zip(v_feats, v_feats_new)), inplace = True)
df.rename(columns = dict(zip(d_feats, d_feats_new)), inplace = True)

In [ ]:
#Validamos el etiquetado
df.columns

### Completitud

Se recomienda que las variables cuenten con una completitud de al menos 80%, de modo que las que no cumplan con esta condición serán eliminadas.

In [ ]:
df.isnull().sum(0)

In [ ]:
#Revisamos la completitud de las columnas del dataset y observamos que aunque no hay peridda de valores menor al 80%, ahora tenemos una variables con 709 valores faltantes
completitud(df)

In [ ]:
#Eliminamos los duplicados
df.dropna(inplace=True)

### Consistencia

En este apartado se realizará la revisión de la naturaleza de las variable, así como algunas modificaciones al dataset inicial que nos será de ayuda para el tratamiento posterior de los datos.

#### Categóricas

In [ ]:
#Revisamos las variables categoricas con la siguiente iteración donde se filtran los valores de las variable de forma única y en un formato de cadena
for i in df.filter(like="v_"):
  print(i)
  values = df[i].astype(str).unique()
  values.sort()
  display(values)
  print("\n")

##### Validación

Después de visualizar de forma general las variables categóricas, se revisará cada una a detalle y se harán los arreglos pertinentes.

**c_passenger_count**
Observamos que hay valores fuera de la naturaleza de las variables, pues no puede haber un servicio de taxi de 0 pasajeros o 208.

In [ ]:
print('\nTotal de datos inconsistentes para la variable \033[1mc_passenger_count\033[0m:')
df[(df['v_passenger_count'] == 0) | (df['v_passenger_count'] == 208)].shape[0]

In [ ]:
#Obtenemos el porcentaje que estos datos invalidos reresentan
print('\nPorcentaje de datos inconsistentes para la variable \033[1mc_passenger_count\033[0m:')
(df[(df['v_passenger_count'] == 0) | (df['v_passenger_count'] == 208)].shape[0]/df.shape[0]) * 100

##### Manipulación de valores
Al revisar las variables continuas se pueden observar las siguientes variables con inconsistencias y sus respectivas causas:

1. **c_passenger_count**: Esta variable representa el número de pasajeros de cada viaje, normalmente los viajes son en autos con máximo 4 lugares y coches mas grandes con hasta 6 lugares, pero hay datos que se encuentran fuera de la naturaleza de la variable; para el caso de esta variable, el porcentaje de datos inválidos es muy pequeño, **0.3550%**, de modo que las eliminamos.

In [ ]:
#Observamos el tamaño del df
df.shape

In [ ]:
#Eliminamos los datos invalidos de la variable c_passenger_count
df = df[(df['v_passenger_count'] != 0) & (df['v_passenger_count'] != 208)]

#Reseteamos el índice
df.reset_index(inplace=True, drop=True)

In [ ]:
#Obtenemos el nuevo tamaño del dataframe
print('El nuevo dataset contienene {} columnas y {} registros.'.format(df.shape[1], "{:,}"
        .format(int(df.shape[0]))))

#Obtenemos el porcentaje de datos perdidos sobre el dataset original
print('El porcentaje de datos perdidos es: {}.'.format((1-df.shape[0]/original_df.shape[0])*100))

#### Continuas

In [ ]:
#Revisamos las variables categoricas con la siguiente iteración donde se filtran los valores de las variable de forma única y en un formato de cadena
for i in df.filter(like="c_"):
  print(i)
  values = df[i].astype(str).unique()
  values.sort()
  display(values)
  print("\n")

##### Validación

Después de visualizar de forma general las variables continuas, se revisará cada una a detalle y se harán los arreglos pertinentes.

**c_fare_amount**

Observamos que hay valores negativos en el precio de los viajes, esto significa que hay valores fuera de la naturaleza de la variable. 

In [ ]:
#Obtenemos la cantidad de registros con precios negativos
print('\nTotal de datos inconsistentes para la variable \033[1mc_fare_amount\033[0m:')
df[df['c_fare_amount'] < 0].shape[0]

In [ ]:
#Obtenemos el porcentaje que estos datos invalidos reresentan
print('\nPorcentaje de datos inconsistentes para la variable \033[1mc_fare_amount\033[0m:')
(df[df['c_fare_amount'] < 0].shape[0]/df.shape[0]) * 100

**pickup_latitude**, **pickup_longitude**, **dropoff_latitude**, **dropoff_longitude**

Observamos que hay valores que no se adecuan a la naturaleza de las variables, pues al ser coordenadas se pueden validar con la meición correcta sel sistema de medición de coornedadas.

In [ ]:
#Visualizamos laa medición correcta de los grados de la tierra con los que tendría que ser válidos en el dataset
Image(url='https://upload.wikimedia.org/wikipedia/commons/thumb/5/58/Latitud_y_Longitud_en_la_Tierra.svg/2880px-Latitud_y_Longitud_en_la_Tierra.svg.png', width=800)

In [ ]:
#Definimos el rango de latituds y longitudes que debería seguir el dataset de acuerdo a la medición de grados de la tierra
print("\nValidamos para las variables con \033[1mlatitudes\033[0m\n")
display(df[(df.c_pickup_latitude>90)])
display(df[(df.c_dropoff_latitude>90)])
display(df[(df.c_pickup_latitude<-90)])
display(df[(df.c_dropoff_latitude<-90)])

#Obtenemos el total de datos invalidos
print('\nTotal de datos inconsistentes para las variables con \033[1mlatitudes\033[0m:')
display((df[(df.c_pickup_latitude>90)].shape[0] + df[(df.c_dropoff_latitude>90)].shape[0] + 
df[(df.c_pickup_latitude<-90)].shape[0] + df[(df.c_dropoff_latitude<-90)].shape[0]))

#Obtenemos el porcentaje que estos datos invalidos reresentan
print('\nPorcentaje de datos inconsistentes para las variables con \033[1mlatitudes\033[0m:')
((df[(df.c_pickup_latitude>90)].shape[0] + df[(df.c_dropoff_latitude>90)].shape[0] + 
df[(df.c_pickup_latitude<-90)].shape[0] + df[(df.c_dropoff_latitude<-90)].shape[0])/df.shape[0]) * 100

In [ ]:
#Definimos el rango de latituds y longitudes que debería seguir el dataset de acuerdo a la medición de grados de la tierra
print("\nValidamos para las variables con \033[1mlongitudes\033[0m\n")
display(df[(df.c_pickup_longitude>180)])
display(df[(df.c_dropoff_longitude>180)])
display(df[(df.c_pickup_longitude<-180)])
display(df[(df.c_dropoff_longitude<-180)])

#Obtenemos el total de datos invalidos
print('\nPorcentaje de datos inconsistentes para las variables con \033[1mlatitudes\033[0m:')
display((df[(df.c_pickup_longitude>180)].shape[0] + df[(df.c_dropoff_longitude>180)].shape[0] + 
df[(df.c_pickup_longitude<-180)].shape[0] + df[(df.c_dropoff_longitude<-180)].shape[0]))

#Obtenemos el porcentaje que estos datos invalidos reresentan
print('\nPorcentaje de datos inconsistentes para las variables con \033[1mlatitudes\033[0m:')
((df[(df.c_pickup_longitude>180)].shape[0] + df[(df.c_dropoff_longitude>180)].shape[0] + 
df[(df.c_pickup_longitude<-180)].shape[0] + df[(df.c_dropoff_longitude<-180)].shape[0])/df.shape[0]) * 100

In [ ]:
#Obtenemos el porcentaje total de los datos inconsistentes para las variables con coordenadas
print('\nPorcentaje de datos inconsistentes para las variables con \033[1mcoordenadas\033[0m:')
((df[(df.c_pickup_latitude>90)].shape[0] + df[(df.c_dropoff_latitude>90)].shape[0] + 
df[(df.c_pickup_latitude<-90)].shape[0] + df[(df.c_dropoff_latitude<-90)].shape[0])/df.shape[0]) * 100 + ((df[(df.c_pickup_longitude>180)].shape[0] + df[(df.c_dropoff_longitude>180)].shape[0] + 
df[(df.c_pickup_longitude<-180)].shape[0] + df[(df.c_dropoff_longitude<-180)].shape[0])/df.shape[0]) * 100

##### Manipulación de valores
Al revisar las variables continuas se pueden observar las siguientes variables con inconsistencias y sus respectivas causas:

1. **c_fare_amount**: Esta variable representa el número de pasajeros de cada viaje, normalmente los viajes son en autos con máximo 4 lugares y coches mas grandes con hasta 6 lugares, pero hay datos que se encuentran fuera de la naturaleza de la variables; para el caso de esta variable, el porcentaje de datos inválidos es muy pequeño, **0.0085%**, de modo que las eliminamos.

2. **pickup_latitude**, **pickup_longitude**, **dropoff_latitude**, **dropoff_longitude**: Las variables que muestran la geolocalización tienen valores fuera de su naturaleza, pues basado en la medición de coordenadas la longitud solo puede medirse entre -180° y 180°, mientras que la latitud entre -90° y 90°; para el caso de estas variable, el porcentaje de datos inválidos es muy pequeño, **0.0085%**, de modo que las eliminamos.

In [ ]:
#Observamos el tamaño del df
df.shape

In [ ]:
#Eliminamos los datos invalidos de la variable c_fare_amount
df = df[df['c_fare_amount'] > 0]

#Eliminamos los datos invalidos de las variables con coordenadas
df = df[(df.c_pickup_latitude<90) & (df.c_dropoff_latitude<90) &
        (df.c_pickup_latitude>-90) & (df.c_dropoff_latitude>-90) &
        (df.c_pickup_longitude<180) & (df.c_dropoff_longitude<180) &
        (df.c_pickup_longitude>-180) & (df.c_dropoff_longitude>-180)]

#Reseteamos el índice
df.reset_index(inplace=True, drop=True)

In [ ]:
#Obtenemos el nuevo tamaño del dataframe
print('El nuevo dataset contienene {} columnas y {} registros.'.format(df.shape[1], "{:,}"
        .format(int(df.shape[0]))))

#Obtenemos el porcentaje de datos perdidos sobre el dataset original
print('El porcentaje de datos perdidos es: {}.'.format((1-df.shape[0]/original_df.shape[0])*100))


#### Fecha

In [ ]:
df['d_pickup_datetime'].describe()

In [ ]:
#Generamos nuevas columnas que nos podrán ayudar al tratamiento de los datos
df['v_year'] = df.d_pickup_datetime.dt.year
df['v_month'] = df.d_pickup_datetime.dt.month
df['v_weekday'] = df.d_pickup_datetime.dt.weekday
df['v_hour'] = df.d_pickup_datetime.dt.hour

In [ ]:
#Creamos categorías segmentadas de trimestre y horarios de los viajes
df['v_quarter'] = df.v_month.map({1:'Q1',2:'Q1',3:'Q1',4:'Q2',5:'Q2',6:'Q2',7:'Q3',
                                      8:'Q3',9:'Q3',10:'Q4',11:'Q4',12:'Q4'})

df['v_hourly_segment'] = df.v_hour.map({0:'H1',1:'H1',2:'H1',3:'H1',4:'H2',5:'H2',6:'H2',7:'H2',8:'H3',
                                     9:'H3',10:'H3',11:'H3',12:'H4',13:'H4',14:'H4',15:'H4',16:'H5',
                                     17:'H5',18:'H5',19:'H5',20:'H6',21:'H6',22:'H6',23:'H6'})

In [ ]:
#Calculamos la distancia de los viajes
df['c_distance']=[round(geopy.distance.distance((df.c_pickup_latitude[i], df.c_pickup_longitude[i]),(df.c_dropoff_latitude[i], df.c_dropoff_longitude[i])).km,2) for i in df.index]

##### Manipulación de valores
Al revisar las variables continuas se pueden observar las siguientes variables con inconsistencias y sus respectivas causas:

1. **d_pickup_datetime**: Debido a que esta variable ha sido dividia en varias columnas que nos dan información más concreta, procedemos a eliminarla.

2. **v_month**: La variable de mes será removida ya que hemos creado una categoría que representa el trimestre.

3. **v_hour**: La variable de hora será removida ya que hemos creado una categoría que representa la hora.

In [ ]:
#Eliminamos la columna de pickup_datetime, month, hour, pues ahora tenemos mejor distribuida la información con las columnas generadas
df.drop(['d_pickup_datetime','v_month', 'v_hour',], axis=1, inplace=True)

#Reseteamos el índice
df.reset_index(inplace=True, drop=True)

In [ ]:
#Visualizamos el nuevo dataset
df.head()

In [ ]:
#Obtenemos el nuevo tamaño del dataframe
print('El nuevo dataset contienene {} columnas y {} registros.'.format(df.shape[1], "{:,}"
        .format(int(df.shape[0]))))

#Obtenemos el porcentaje de datos perdidos sobre el dataset original
print('El porcentaje de datos perdidos es: {}.'.format((1-df.shape[0]/original_df.shape[0])*100))

### Completitud

Se recomienda que las variables cuenten con una completitud de al menos 80%, de modo que las que no cumplan con esta condición serán eliminadas.

In [ ]:
df.isnull().sum(0)

In [ ]:
#Revisamos la completitud de las columnas del dataset y observamos que aunque no hay peridda de valores menor al 80%, ahora tenemos una variables con 709 valores faltantes
completitud(df)

### Duplicidad

#### General

In [ ]:
#Total de registros duplicados de forma general
df.duplicated().sum()

In [ ]:
print('\nPorcentaje de datos \033[1mduplicados\033[0m:')
(df.duplicated().sum()/df.shape[0]) * 100

In [ ]:
df[df.duplicated()]

In [ ]:
#Eliminamos los duplicados al representar un porcentaje bajo de registros y mantenemos el primer elemento de los duplicados
df.drop_duplicates(keep = 'first', inplace = True)

#Reseteamos el índice
df.reset_index(inplace=True, drop=True)

In [ ]:
#Validamos la eliminacipon de duplicados
df[df.duplicated()]

In [ ]:
#Obtenemos el nuevo tamaño del dataframe
print('El nuevo dataset contienene {} columnas y {} registros.'.format(df.shape[1], "{:,}"
        .format(int(df.shape[0]))))

#Obtenemos el porcentaje de datos perdidos sobre el dataset original
print('El porcentaje de datos perdidos es: {}.'.format((1-df.shape[0]/original_df.shape[0])*100))

##### Observaciones

En esta ocasión, en el presente dataset, ninguna de las columnas contó con una completitud menor al umbral definido.

## **Exploratory Data Analysis (EDA)**
---

In [ ]:
#Observamos el dataset
df.head()

In [ ]:
#Obtenemos la información
df.info()

In [ ]:
#Obtenemos el tamaño del dataset
df.shape

In [ ]:
#Cambiamos los tipos de datos para una mejor manipulación
df["v_passenger_count"] = df["v_passenger_count"].astype("string")
df["v_year"] = df["v_year"].astype("string")
df["v_weekday"] = df["v_weekday"].astype("string")
df["v_hourly_segment"] = df["v_hourly_segment"].astype("string")
df["v_quarter"] = df["v_quarter"].astype("string")

#Añadimos una columna que nos ayudará con los gráficos
df["count"] = 1

### Target

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable target
fig = px.histogram(df["c_fare_amount"], nbins=15, title='Gráfico de caja e histograma de la variable target: c_fare_amount',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_fare_amount')
fig.show()

In [ ]:
df["c_fare_amount"].describe().T

#### Conclusiones

**Gráfico de cajas**: Podemos ver los outliers, al menos el valor máximo (499 USD) puede observarse bastante alejado del resto en el gráfico de cajas. Este así como el resto deberán eliminarse más adelante.

**Histograma**: Podemos observar que la distribución no parece comportarse como una normal.

**Estadísticos**

**Promedio**: Obtuvimos un promedio de 11.3714 USD para el precio de los viajes.  

**Desviación estandar**: La variablidad de los datos con respecto al promedio es de 9.9081.  

**Mínimo**: El valor mínimo de un viaje es 0.0100 centavos de USD.  

**Máximo** 499.0000.  

**Cuartiles**  

**25%** -> 6.0000  
**50%** -> 8.5000  
**75%** -> 12.5000  

### Continuas

In [ ]:
#Visualizamos descriptivos 
df[["c_pickup_longitude", 
    "c_pickup_latitude", 
    "c_dropoff_longitude",
    "c_dropoff_latitude", 
    "c_distance"]].describe().T

Variables de recolección del pasaje

**c_pickup_longitude**, **c_pickup_latitude**.

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable target
fig = px.histogram(df["c_pickup_longitude"], nbins=15, title='Gráfico de caja e histograma de la variable c_pickup_longitude',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_pickup_longitude')
fig.show()

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable target
fig = px.histogram(df["c_pickup_latitude"], nbins=15, title='Gráfico de caja e histograma de la variable c_pickup_latitude',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_pickup_latitude')
fig.show()

In [ ]:
#Gráficamos los lugares de recolección en el mundo que contiene el dataset
fig = px.scatter_geo(df,
                    lat=df.c_pickup_latitude,
                    lon=df.c_pickup_longitude)
fig.update_layout(
    title = "Países de recolección de pasaje")
fig.show()

In [ ]:
#Gráficamos los lugares de recolección en el país que cuenta con más registros en el dataset (EE.UUU)
fig = px.scatter_geo(df,
                    lat=df.c_pickup_latitude,
                    lon=df.c_pickup_longitude)
fig.update_layout(
    title = "Lugares de recolección de pasaje en EE.UU<br>(El país con más registros en el dataset)",
    geo_scope = "usa")
fig.show()

Variables de desocupación del pasaje

**c_dropoff_longitude**, **c_dropoff_latitude**.

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable target
fig = px.histogram(df["c_dropoff_longitude"], nbins=15, title='Gráfico de caja e histograma de la variable c_dropoff_longitude',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_dropoff_longitude')
fig.show()

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable target
fig = px.histogram(df["c_dropoff_latitude"], nbins=15, title='Gráfico de caja e histograma de la variable c_dropoff_latitude',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_dropoff_latitude')
fig.show()

In [ ]:
#Gráficamos los lugares de recolección en el mundo que contiene el dataset
fig = px.scatter_geo(df,
                    lat=df.c_dropoff_latitude,
                    lon=df.c_dropoff_longitude)
fig.update_layout(
    title = "Países de desocupación de pasaje")
fig.show()

In [ ]:
#Gráficamos los lugares de recolección en el mundo que contiene el dataset
fig = px.scatter_geo(df,
                    lat=df.c_dropoff_latitude,
                    lon=df.c_dropoff_longitude)
fig.update_layout(
    title ="Lugares de recolección de pasaje en EE.UU<br>(El país con más registros en el dataset)"
    geo_scope = "usa")
fig.show()

**c_distance**

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable target
fig = px.histogram(df["c_distance"], nbins=15, title='Gráfico de caja e histograma de la variable c_distance',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_distance')
fig.show()

#### Conclusiones

**Gráficos espacial**: Podemos observar que la distribución de los puntos de recolección y desocupación del pasaje están centrados en EE.UU., y hay algunos puntos extraños en el mar y en la Antártida, valores que esperamos eliminar más adelante.

**Gráfico de cajas**: Para el caso de la variable distancia, podemos observar la presencia de outliers, siendo el mayor oulier el máximo de distancia de viaje, el cual esta representado por **8,783.59** KM.

**Histograma**: Podemos observar que la distribución no parece comportarse como una normal.

### Categóricas

In [ ]:
#Obtenemos las variables categóricas del dataset
for i in df.filter(like="v_"):
  print(i)

**v_passenger_count**

Podemos observar que norlmalmente los viajes son de **1** pasajero, representando poco más de **138K** viajes, lo que represento el **69.4%**.

In [ ]:
fig = px.bar(pd.DataFrame(df.groupby(["v_passenger_count"])["count"].count()).reset_index(), x = 'v_passenger_count', y="count", title="Frecuencia por tipo de viajes por número de pasajeros", color ="v_passenger_count")
fig.update_layout(yaxis_title='Count', xaxis_title='c_fare_amount')
fig.show()

In [ ]:
fig = px.pie(df, values='count', names='v_passenger_count', title='Porcentaje de tipo de viajes por número de pasajeros')
fig.show()

**v_year**

Podemos observar que el año con más viajes registrados es **2012**, respresentando poco más de **30K** viajes, lo que representó el **16.1%** del total de viajes registrados.

In [ ]:
fig = px.bar(pd.DataFrame(df.groupby(["v_year"])["count"].count()).reset_index(), x = 'v_year', y="count", title="Frecuencia de viajes por año", color ="v_year")
fig.update_layout(yaxis_title='Count', xaxis_title='v_year')
fig.show()

In [ ]:
fig = px.pie(df, values='count', names='v_year', title='Porcentaje de viajes por año')
fig.show()

**v_weekday**

Podemos observar que el día con más viajes registrados es el **viernes**, respresentando poco más de **30K** viajes, lo que representó el **16.2%** del total de viajes registrados.

In [ ]:
fig = px.bar(pd.DataFrame(df.groupby(["v_weekday"])["count"].count()).reset_index(), x = 'v_weekday', y="count", title="Frecuencia de viajes por día", color ="v_weekday")
fig.update_layout(yaxis_title='Count', xaxis_title='v_weekday')
fig.show()

In [ ]:
fig = px.pie(df, values='count', names='v_weekday', title="Porcentaje de viajes por día")
fig.show()

**v_quarter**

Podemos observar que el trimestre con más viajes registrados es el **Q2**, respresentando poco más de **55K** viajes, lo que representó el **27.6%** del total de viajes registrados.

In [ ]:
fig = px.bar(pd.DataFrame(df.groupby(["v_quarter"])["count"].count()).reset_index(), x = 'v_quarter', y="count", title="Frecuencia de viajes por trimestre", color ="v_quarter")
fig.update_layout(yaxis_title='Count', xaxis_title='v_quarter')
fig.show()

In [ ]:
fig = px.pie(df, values='count', names='v_quarter', title="Porcentaje de viajes por trimestre")
fig.show()

**v_quarter**

Podemos observar que el segmento de horas con más viajes registrados es el **H6**, es decir, entre las 20:00 y 23:00 horas, respresentando poco más de **43K** viajes, lo que representó el **22%** del total de viajes registrados.

In [ ]:
fig = px.bar(pd.DataFrame(df.groupby(["v_hourly_segment"])["count"].count()).reset_index(), x = 'v_hourly_segment', y="count", title="Frecuencia de viajes por segmento de hora", color ="v_hourly_segment")
fig.update_layout(yaxis_title='Count', xaxis_title='v_hourly_segment')
fig.show()

In [ ]:
fig = px.pie(df, values='count', names='v_hourly_segment', title="Porcentaje de viajes por segmento de hora")
fig.show()

## **Datos anómalos**
---

### Histogramas antes de la remoción de outliers

#### Target

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable target
fig = px.histogram(df["c_fare_amount"], nbins=15, title='Gráfico de caja e histograma de la variable target: c_fare_amount',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_fare_amount')
fig.show()

#### Variables de recolección del pasaje

**c_pickup_longitude**, **c_pickup_latitude**.

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable c_pickup_longitude
fig = px.histogram(df["c_pickup_longitude"], nbins=15, title='Gráfico de caja e histograma de la variable c_pickup_longitude',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_pickup_longitude')
fig.show()

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable c_pickup_latitude
fig = px.histogram(df["c_pickup_latitude"], nbins=15, title='Gráfico de caja e histograma de la variable c_pickup_latitude',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_pickup_latitude')
fig.show()

#### Variables de desocupación del pasaje

**c_dropoff_longitude**, **c_dropoff_latitude**.

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable c_dropoff_longitude
fig = px.histogram(df["c_dropoff_longitude"], nbins=15, title='Gráfico de caja e histograma de la variable c_dropoff_longitude',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_dropoff_longitude')
fig.show()

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable c_dropoff_latitude
fig = px.histogram(df["c_dropoff_latitude"], nbins=15, title='Gráfico de caja e histograma de la variable c_dropoff_latitude',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_dropoff_latitude')
fig.show()

#### **c_distance**

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable c_distance
fig = px.histogram(df["c_distance"], nbins=15, title='Gráfico de caja e histograma de la variable c_distance',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_distance')
fig.show()

### Eliminación de outliers

**Método**

Debido a la naturaleza de nuestras variables, las cuales no siguen un distribución normal, utilizaremos el método univariado de percentiles.

In [ ]:
#Aplicamos la función de OUTLIERS la cual muestra un resuen de los diversos métodos, debido a las pruebas realizadas anteriormente, solo mostraremos atención en la prueba de los percentiles
outliers=OUTLIERS(df,list(df.filter(like="c_")))
outliers

In [ ]:
#Guardamos en una lista los indices identificados como atípicos
indices = list(outliers["indices"].values)

In [ ]:
#Guardamos los indicies que nos interesan
indices = list(set(reduce(lambda x,y: x+y, indices)))

In [ ]:
#Obtenemos el porcentaje de elementos que eliminaremos
(len(indices)/df.shape[0])*100

In [ ]:
#Mostramos los valores a imputar
df[df.index.isin(indices)]

In [ ]:
#Obtenemos el tamaño del actual dataframe
df.shape

In [ ]:
df = df[~df.index.isin(indices)].reset_index(drop=True)
df.shape

In [ ]:
#Obtenemos el nuevo tamaño del dataframe
print('El nuevo dataset contienene {} columnas y {} registros.'.format(df.shape[1], "{:,}"
        .format(int(df.shape[0]))))

#Obtenemos el porcentaje de datos perdidos sobre el dataset original
print('El porcentaje de datos perdidos es: {}.'.format((1-df.shape[0]/original_df.shape[0])*100))

### Histogramas antes de la remoción de outliers

#### Target

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable target
fig = px.histogram(df["c_fare_amount"], nbins=15, title='Gráfico de caja e histograma de la variable target: c_fare_amount',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_fare_amount')
fig.show()

#### Variables de recolección del pasaje

**c_pickup_longitude**, **c_pickup_latitude**.

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable c_pickup_longitude
fig = px.histogram(df["c_pickup_longitude"], nbins=15, title='Gráfico de caja e histograma de la variable c_pickup_longitude',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_pickup_longitude')
fig.show()

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable c_pickup_latitude
fig = px.histogram(df["c_pickup_latitude"], nbins=15, title='Gráfico de caja e histograma de la variable c_pickup_latitude',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_pickup_latitude')
fig.show()

#### Variables de desocupación del pasaje

**c_dropoff_longitude**, **c_dropoff_latitude**.

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable c_dropoff_longitude
fig = px.histogram(df["c_dropoff_longitude"], nbins=15, title='Gráfico de caja e histograma de la variable c_dropoff_longitude',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_dropoff_longitude')
fig.show()

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable c_dropoff_latitude
fig = px.histogram(df["c_dropoff_latitude"], nbins=15, title='Gráfico de caja e histograma de la variable c_dropoff_latitude',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_dropoff_latitude')
fig.show()

#### **c_distance**

In [ ]:
#Realizamos un gráfico de cajas y un histograma para revisar a la variable c_distance
fig = px.histogram(df["c_distance"], nbins=15, title='Gráfico de caja e histograma de la variable c_distance',marginal="box")
fig.update_layout(yaxis_title='count', xaxis_title='c_distance')
fig.show()

## **Datos faltantes**
---

In [ ]:
#Revisamos nuevamente los datos faltantes
completitud(df)

## **Ingenieria de variables**
---

### Consideraciones

Debido a la naturaleza de las variables **continuas** las cuales son:

- **c_fare_amount**: Variable de precio objetivo.

- **c_pickup_longitude**: Variable de geolocalización.

- **c_pickup_latitude**: Variable de geolocalización.

- **c_dropoff_longitude**: Variable de geolocalización.

- **c_dropoff_latitude**: Variable de geolocalización.

- **c_distance**: Distancia en KM.

Considero que no es no necesario aplicar ningún método de ingenieria de variables, no es así para el caso de las varibles **categóricas**, las cuales son:

- **v_passenger_count**

- **v_year**

- **v_weekday**

- **v_quarter**

- **v_hourly_segment**

En estas últimas aplicaremos ingeniería de variables.

### One-Hot Encoding

In [ ]:
#Realizamos la creación de las variables dummy
df = pd.get_dummies(df,columns=['v_passenger_count',"v_year", 
                                "v_weekday", "v_quarter", 
                                "v_hourly_segment"], drop_first=True, prefix_sep='_')

In [ ]:
df.head()

### Count Encoding

**Conjunto de entrenamiento y prueba**

Definimos el conjunto de entrenamiento con el 70% y el de prueba con el 30%.

In [ ]:
#Definimos los conjuntos
X_train, X_test = train_test_split(df, test_size = 0.30, random_state = 413)

In [ ]:
X_train["v_residence_type"].value_counts(1)

In [ ]:
X_test["v_residence_type"].value_counts(1)